## One-Sample T-Test
A one-sample t-test checks whether a sample mean differs from the population mean. Let's create some dummy data for the population and a sample and test whether the average value of the sample differs from the average value of the population.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import math

In [2]:
np.random.seed(8)
population = stats.poisson.rvs(loc=0, mu=50, size=100000)
sample = stats.poisson.rvs(loc=0, mu=48, size=100)

We used a slightly different distribution to generate the sample data, so we know that the two means are different. Let's conduct a t-test at a 95% confidence level and see if it correctly rejects the null hypothesis that the sample comes from the same distribution as the population. To conduct a one sample t-test, we use the stats.ttest_1samp() function.

In [3]:
t_stat, p_val = stats.ttest_1samp(a=sample, popmean=population.mean())

print('t-statistic: {0}'.format(round(t_stat, 2)))
print('p-value: {0}'.format(round(p_val, 2)))

t-statistic: -2.59
p-value: 0.01


The test result shows the test statistic is equal to -2.59. This test statistic tells us how much the sample mean deviates from the null hypothesis. If the t-statistic lies outside the quantiles of the t-distribution corresponding to our confidence level and degrees of freedom, we reject the null hypothesis. We can check the quantiles with stats.t.ppf().

In [4]:
t_critical = stats.t.ppf(q=0.975, df=99)

print('t-critical value: {0}'.format(round(t_critical, 2)))

t-critical value: 1.98


We can calculate the chances of seeing a result as extreme as the one we observed (known as the p-value) by passing the t-statistic in as the quantile to the stats.t.cdf() function.

In [5]:
print('p-value: {0}'.format(round(stats.t.cdf(x=t_stat, df=99)*2, 2)))

p-value: 0.01


The alternative hypothesis we are checking is whether the sample mean differs (is not equal to) the population mean. Since the sample could differ in either the positive or negative direction we multiply by two.
Notice this value is the same as the p-value listed in the original t-test output. A p-value of 0.01 means we'd expect to see data as extreme as our sample due to chance about 1% of the time if the null hypothesis was true. In this case, the p-value is lower than our significance level α (equal to 1 - confidence level or 0.05) so we should reject the null hypothesis. If we were to construct a 95% confidence interval for the sample it would not capture population mean of 50.

In [6]:
sigma = sample.std()/math.sqrt(100)
left_val, right_val = stats.t.interval(0.95, df=99, loc=sample.mean(), scale=sigma)

print('CI: {0}, {1}'.format(round(left_val, 2), round(right_val, 2)))

CI: 46.39, 49.49


On the other hand, since there is a 1.3% chance of seeing a result this extreme due to chance, it is not significant at the 99% confidence level. This means if we were to construct a 99% confidence interval, it would capture the population mean.

In [7]:
sigma = sample.std()/math.sqrt(100)
left_val, right_val = stats.t.interval(0.99, df=99, loc=sample.mean(), scale=sigma)

print('CI: {0}, {1}'.format(round(left_val, 2), round(right_val, 2)))

CI: 45.89, 49.99


With a higher confidence level, we construct a wider confidence interval and increase the chances that it captures to true mean, thus making it less likely that we'll reject the null hypothesis. In this case, the p-value of 0.0109 is greater than our significance level of 0.01 and we fail to reject the null hypothesis.

## Two-Sample T-Test
A two-sample t-test investigates whether the means of two independent data samples differ from one another. In a two-sample test, the null hypothesis is that the means of both groups are the same. Unlike the one sample-test where we test against a known population parameter, the two sample test only involves sample means. You can conduct a two-sample t-test by passing with the stats.ttest_ind() function. Let's generate two data samples and test it against each other.

In [8]:
np.random.seed(8)
first_sample = stats.poisson.rvs(loc=0, mu=30, size=20)
second_sample = stats.poisson.rvs(loc=0, mu=25, size=30)

In [9]:
t_stat, p_val = stats.ttest_ind(a=first_sample, b=second_sample, equal_var=False)

print('t-statistic: {0}'.format(round(t_stat, 2)))
print('p-value: {0}'.format(round(p_val, 2)))

t-statistic: 2.4
p-value: 0.02


The test yields a p-value of 0.02, which means there is a 2% chance we'd see sample data this far apart if the two groups tested are actually identical. If we were using a 99% confidence level we would fail to reject the null hypothesis, since the p-value is greater than the corresponding significance level of 1%.

## Paired T-Test

The basic two sample t-test is designed for testing differences between independent groups. In some cases, you might be interested in testing differences between samples of the same group at different points in time. For instance, a hospital might want to test whether a drug works by checking the parameters of the same group of patients before and after treatment. A paired t-test lets you check whether the means of samples from the same group differ.
We can conduct a paired t-test using the scipy function stats.ttest_rel(). Let's generate some data and do a paired t-test.

In [10]:
np.random.seed(8)
before_treatment = stats.norm.rvs(scale=10, loc=200, size=100)
after_treatment = before_treatment + stats.norm.rvs(scale=40, loc=10, size=100)
difference = after_treatment - before_treatment

df = pd.DataFrame({'Before treatment': before_treatment, 
                   'After treatment': after_treatment, 
                   'Difference': difference})

print('Mean difference: {0}'.format(round(df.Difference.mean(), 2)))

Mean difference: 8.58


The summary shows that the mean difference is about 8 on average after treatment. Let's conduct a paired t-test to see whether this difference is significant at a 95% confidence level.

In [11]:
t_stat, p_val = stats.ttest_rel(a=before_treatment, b=after_treatment)
print('t-statistic: {0}'.format(round(t_stat, 2)))
print('p-value: {0}'.format(round(p_val, 2)))

t-statistic: -1.95
p-value: 0.05


The p-value in the test output shows that the chances of seeing this large of a difference between samples due to chance is 5%.